In [1]:
import pandas as pd
import random
from tqdm.notebook import tqdm
from random import sample

In [2]:
# Data sebagai bahan baku u/ generate kalimat order
df_bahan = pd.read_excel("../data/resource/bahan baku bikin kalimat.xlsx")

# Dictionary u/ convert kata ke bentuk spelling-nya
df_conv_spell = pd.read_excel("../data/resource/dict_conv_spelling.xlsx")

# Dictionary kata-kata pada orderan yg sudah dikategorikan berdasarkan label entity
df_kata = pd.read_excel("../data/resource/dict_order.xlsx")

In [3]:
'''Untuk setiap product & uom variant dipisahkan dengan (@) '''
product = df_bahan['product variant'].dropna()
uom = df_bahan['uom variant'].dropna()
numeric = df_bahan.numeric.dropna()

'''Untuk setiap kata dan posibility spelling dipisahkan dengan (@)'''
word = df_conv_spell.word.dropna()
spell = df_conv_spell.spell.dropna()

In [4]:
list_product = []
list_product_delimiter = []
list_word = []


list_product_delimiter = product.values.tolist() # it's contain '@' as delimiter
    
product = product[product!='@'] # remove '@'
product = product.reset_index(drop=True)
list_product = product.values.tolist() # it's doesn't have '@' as delimiter

word = word[word!='@']
word = word.reset_index(drop=True)
list_word = word.values.tolist()

In [5]:
join_product = ','.join(list_product_delimiter)
join_spell = ','.join(spell)
join_uom = ','.join(uom)

In [6]:
spell = join_spell.split('@')
uom = join_uom.split('@')
prod = join_product.split('@')

spell_word = [i.strip(',') for i in spell]
uom_word = [i.strip(',') for i in uom]
prod_word = [i.strip(',') for i in prod]

In [7]:
# Total product which have this UOM type
length_product = [len(data.split(',')) for data in prod_word]

In [8]:
## New UOM
new_uom = []

for idx in range (len(length_product)):
    for i in range (length_product[idx]):
        new_uom.append(uom_word[idx])

In [9]:
'''
    Combine product + numeric + uom
    which is product already translate to spelling format
'''
data = []

for line in list_product:        
    status = False
    for word in list_word:
        if word in line:
            status = True
            after_word = spell_word[list_word.index(word)]
            temp_uom = new_uom[list_word.index(word)]
            
            for val_spell in after_word.split(','):
                for val_uom in temp_uom.split(','):
                    for num in numeric:
                        data.append(line.replace(word, val_spell)+' '+num+' '+val_uom)
                        
    if status == False:
        temp_uom = new_uom[list_product.index(line)]
        for val_uom in temp_uom.split(','):
            for num in numeric:
                data.append(line+' '+num+' '+val_uom)
                
data = [' '.join(line.strip().split()) for line in data]

In [10]:
# For make sure that all word already translate to spelling format
for i in range(3):
    for idx in range(0,len(data)):
        for word in list_word:
            if word in data[idx]:
                after_word = spell_word[list_word.index(word)]
            
                for val_spell in after_word.split(','):
                    data[idx] = data[idx].replace(word, val_spell)

In [11]:
random.choice(data)

'enerjen sachet coklat juh lembar'

In [12]:
# Save data as file text
with open('../data/train/artificial_data.txt','w') as outfile:
    for line in data:
        outfile.write('%s\n' % line)